1. Building a Data Pipeline

In [ ]:
import tensorflow as tf 
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
data_dir = 'data'

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
# This would be used for image cleanup:
def image_cleanup():
    for image_class in os.listdir(data_dir):
        for image in os.listdir(os.path.join(data_dir, image_class)):
            image_path = os.path.join(data_dir, image_class, image)
            try:
                img = cv2.imread(image_path)
                tip = imghdr.what(image_path)
                if tip not in image_exts:
                    print('Image not in ext list {}'.format(image_path))
            except Exception as e:
                print('Issue with image {}'.format(image_path))
#image_cleanup()

In [ ]:
#Test file read and plot
img = cv2.imread(os.path.join('data','Quarter-Note', '9-95_3.png'))
plt.imshow(img)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')
for batch_images, batch_labels in data.take(1):
    # Print the shape of the batch images
    print("Batch Images Shape:", batch_images.shape)
    # Print the shape of the batch labels
    print("Batch Labels Shape:", batch_labels.shape)

2. Pre-process Data

In [ ]:
input_dir = 'data'
output_dir = 'processed'

desired_size = 256

# Traverse through subfolders and process images
for label_name in os.listdir(input_dir):
    label_dir = os.path.join(input_dir, label_name)
    output_label_dir = os.path.join(output_dir, label_name)
    os.makedirs(output_label_dir, exist_ok=True)
    
    for image_name in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_name)
        output_image_path = os.path.join(output_label_dir, image_name)
        
        # Load the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Resize while maintaining aspect ratio
        aspect_ratio = image.shape[1] / image.shape[0]
        new_width = int(desired_size * aspect_ratio)
        resized_image = cv2.resize(image, (new_width, desired_size))
        
        # Calculate padding
        padding_needed = desired_size - new_width
        left_padding = max(padding_needed // 2, 0)
        right_padding = max(padding_needed - left_padding, 0)
        
        # Pad the image
        padded_image = cv2.copyMakeBorder(resized_image, 0, 0, left_padding, right_padding, cv2.BORDER_CONSTANT, value=255)
        
        # Create an RGB image
        final_image = np.stack((padded_image,) * 3, axis=-1)
        
        # Save the preprocessed image
        cv2.imwrite(output_image_path, final_image)

In [ ]:
pre_processed = tf.keras.utils.image_dataset_from_directory('processed')
scaled_data = pre_processed.map(lambda x, y: (x / 255, y))

In [ ]:
train_size = int(len(scaled_data)*.7)
val_size = int(len(scaled_data)*.2)
test_size = int(len(scaled_data)*.1)

In [ ]:
train = scaled_data.take(train_size)
val = scaled_data.skip(train_size).take(val_size)
test = scaled_data.skip(train_size+val_size).take(test_size)

3. Deep Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
# Add convolutional layers
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add a fully connected layer
model.add(Dense(units=128, activation='relu'))

# Add the output layer
num_classes = 32  # Update with the number of classes (symbols) in your OMR task
model.add(Dense(units=num_classes, activation='softmax'))

In [ ]:
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='loss')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_loss')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

4. Evaluate Performance

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    predicted_classes = np.argmax(yhat, axis=1)
    print(X)
    print(to_categorical(predicted_classes))
    pre.update_state(y, predicted_classes)
    re.update_state(y, predicted_classes)
    acc.update_state(y, predicted_classes)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
img = cv2.imread('./processed/Natural/3-89_3.png')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256, 256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
resize.shape

In [ ]:
# Add an extra dimension to fool the model into thinking it's a batch
np.expand_dims(resize, 0)

In [ ]:
yhat_test = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
# TODO: Figure out how to actually print which class it predicted? Cuz like rn it's just predicting integers instead of strings
data_dir = 'data/processed'
class_names = sorted(os.listdir(data_dir))
predictions = yhat_test
print(predictions)
print(predictions.shape)
print(np.argmax(predictions))
print(class_names[np.argmax(predictions)])
all_predictions = predictions[0].tolist()
predicted_with_confidence = [(class_names[i], confidence) for i, confidence in enumerate(all_predictions)]
predicted_with_confidence_sorted = sorted(predicted_with_confidence, key=lambda x: x[1], reverse=True)
for i in range(4):
    note_name, confidence = predicted_with_confidence_sorted[i]
    print(f"Note: {note_name}, Confidence: {confidence:.4f}")

5. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','amadeus_model.keras'))

In [ ]:
new_model = load_model(os.path.join('models','amadeus_model.keras'))

In [ ]:
yhat_new = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
# TODO: Figure out how to actually print which class it predicted? Cuz like rn it's just predicting integers instead of strings
to_categorical(np.argmax(yhat_new, axis=1))